# Density matrix exponentiation

This is a function to simulate DME. It follows the following protocol:
![image.png](attachment:image.png)
Goals: implement the unitary $U=e^{-i\rho\theta}$ on data qubit $\sigma$ (rotate $\sigma$ by $\theta$) according to instruction given by instruction qubit $\rho$.

### Function `swap_rotation`
First, DME uses the relation (rotate $\sigma$ by small angle $\delta = \sigma/N$).
$$\begin{align}
Tr_\rho [e^{-iSWAP\delta}\sigma\otimes\rho e^{iSWAP\delta}] &= \sigma - i\delta[\rho,\sigma] +\mathcal{O}(\delta^2)\\
&= e^{-i\rho\delta}\sigma e^{i\rho\delta}+\mathcal{O}(\delta^2)
\end{align}$$


### Function `QME`
Second, DME uses either a simple reset, or QME to dynamically reset the instruction qubit.

QME apply $\mathbb{1}$, or $\pi$-rotation in the instruction state basis $R_\nu(\pi)$ with probability $p = 0.5$

Outcomes of multiple such randomized instantiations are averaged $\to$ mimic single circuit with active reset of $\rho$

### Function `DME_evolution`
Combine the above, this function execute the above circuit (without QME) or the following circuit (with QME):

![image-2.png](attachment:image-2.png)


In [ ]:
import numpy as np
from qutip.qip.gates import swap
from qibo.config import raise_error

def test_qdp_memory_usage_circuit():
    qdp = quantum_dynamic_programming(1,10)
    import qibolab
    platform = get_platform("")
    opt = Options(nshots = 1000)
    platform.run(qdp.memory_usage_circuit())

class QDP_memory_type(Enum):
    from enum import Enum, auto
    default = auto()
    reset = auto()
    quantum_measurement_emulation = auto()

class quantum_dynamic_programming:
    def __init__(self, num_work_qubits, num_instruction_qubits, number_muq_per_call):
        self.N = num_work_qubits,
        self.id_work_reg = np.arrange(0,num_work_qubits,1)
        self.id_instruction_reg = np.arrange(0,num_instruction_qubits,1)
        self.id_current_instruction_reg = 0
        self.M = number_muq_per_call
        self.memory_type = QDP_memory_type.default
    
    def __call__(self,instruction_reg):
        c = self.memory_call_circuit()
        # for now we assume that the recursion step make only 1 type memory call per QDP step

    def memory_call_circuit(self):
        c = Circuit(num_working + num_reg)
        if self.memory_type == QDP_memory_type.default:
            for register in self.id_instruction_reg[self.get_current_instruction_register():self.M+self.get_current_instruction_register()]:
                c.add(self.memory_usage_circuit(register))

    def get_current_instruction_register(self):
        return self.id_current_instruction_reg
    
    def increment_current_instruction_register(self):
        self.id_current_instruction_reg += 1

    @abstractmethod
    def memory_usage_query(self,N_op,rho,sigma = None):
        if sigma == None:
            sigma = self.sigma
        return trace_1(np.exp(-1j*N_op) * tensor(rho,sigma) * np.exp(1j*N_op))
        
    #todo look up qibo evolution
    @abstractmethod
    def memory_usage_circuit(self,ind_work_reg):
        c = Circuit(self.num_working + self.num_instruction)
        c.add(gates.swap(0,1))
        c.add(gates.M(0))
        c.add(gates.M(1))
        return c

In [ ]:
class quantum_dynamic_programming_numpy:
    def __init__(self, sigma):
        self.sigma = sigma

    @abstractmethod
    def memory_usage_query(self,N_op,rho,sigma = None):
        if sigma == None:
            sigma = self.sigma
        return trace_1(np.exp(-1j*N_op) * tensor(rho,sigma) * np.exp(1j*N_op))
        
    #todo look up qibo evolution

    def memory_call(self):
        raise_error(NotImplementedError)



class density_matrix_exponentiation(quantum_dynamic_programming):
    def __init__(self, rho, sigma, theta, N):
        self.rho = rho # instruction qubit
        self.sigma = sigma # data_qubit
        self.theta = theta # overall rotation angle
        self.N = N # number of steps
        self.delta = theta/N # small rotation angle
        self.partial_swap_negative = np.exp(-1j*swap()*self.delta)
        self.partial_swap_positive = np.exp(1j*swap()*self.delta)

    def DME_evolution(self):
        current_sigma, current_rho = self.sigma, self.rho
        for _ in range(self.N):
            current_sigma, current_rho = self.swap_rotation(current_sigma, current_rho)
        return current_sigma, current_rho

    def swap_rotation(self, current_sigma, current_rho, QME = False):        
        system = tensor(current_sigma, current_rho)
        unitary_evolution = self.partial_swap_negative * system * self.partial_swap_positive
        new_sigma = trace_rho(unitary_evolution)
        if QME == True:
            new_rho = self.QME(current_rho)
        elif QME == False:
            new_rho = self.rho 
        return new_sigma, new_rho

    def QME(self, current_rho):
        import random
        coin_flip = random.choice([0, 1])
        if coin_flip == 0:
            QME_gate = R_nu(np.pi) # nu is the vector parallel to rho
        elif coin_flip == 1:
            QME_gate = identity()
        new_rho = QME_gate * current_rho
        return new_rho

    def measurement(self):
        # measurement or state tomography
        current_sigma, current_rho = self.DME_evolution()
        final_sigma = measurement_gate * current_sigma
        final_rho = measurement_gate * current_rho
        return final_sigma, final_rho